In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import rasterio
import sys
sys.path.append("..")

In [ ]:
from power_planner.utils.utils import *
from power_planner.data_reader import *

In [ ]:
path_files = "../../data"

###  Data exploration / corridor save as json

In [ ]:
with rasterio.open(os.path.join(path_files, "corridor/Corridor_BE.tif"), 'r') as ds:
    arr = ds.read()
corr_img = Image.fromarray(arr[0])
plt.hist(np.asarray(corr_img).flatten())
# plt.imshow(normalize(corr_img))
# plt.colorbar()
plt.show()

In [ ]:
import json
with open("../outputs/path_52421_infos.json", "r") as infile:
    path = json.load(infile)["path_coordinates"]

In [ ]:
from rasterio import features

In [ ]:
    # Extract feature shapes and values from the array.
    for geom, val in features.shapes(
            mask, transform=dataset.transform):

        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326).
        geom = rasterio.warp.transform_geom(
            dataset.crs, 'EPSG:4326', geom, precision=6)

        # Print GeoJSON shapes to stdout.
        print(geom)

In [ ]:
with rasterio.open(os.path.join(path_files, "corridor/COSTSURFACE.tif")) as dataset:

    # Read the dataset's valid data mask as a ndarray.
    mask = dataset.dataset_mask() # gives the 0-1 mask I need!
    print(mask.shape)
    print(dataset.width)
    print(dataset.bounds)
    transform_matrix = dataset.transform
    arr = dataset.read()
    print(arr.shape)
    print(dataset.crs)

In [ ]:
crs = rasterio.crs.CRS()

In [ ]:
crs = rasterio.crs.CRS.from_dict(init='epsg:31370')

In [ ]:
crs.is_epsg_code

In [ ]:
coordinates = [transform_matrix*tuple(p) for p in path]
print(coordinates)

In [ ]:
with open("../outputs/path_result_whole_instance_coords.json", "w") as outfile:
    json.dump(coordinates, outfile)

## Path straightening:

In [ ]:
plot_path(normalize(arr[0]), path, out_path="corridor_figure.png")

In [ ]:
path = np.asarray(path)

In [ ]:
from rdp import rdp, rdp_rec

In [ ]:
straight = rdp_rec(path, epsilon=2) # eps from 0.3 to 2 tested

In [ ]:
len(path),len(straight)

In [ ]:
plt.scatter(path[:,0], path[:,1])
plt.scatter(straight[:,0], straight[:,1])
plt.show()

### RDP Observations:

* The Ramer-Douglas-Peucker algorithm is an algorithm for reducing the number of points in a curve that is approximated by a series of points.
* leaves curves, simply reduces number of points

## Check plotting

In [ ]:
arr = plt.imread("../outputs/path_02182.png")

In [ ]:
np.all(arr[0,0] == np.array([1., 1, 1, 1]))

In [ ]:
arr2 = arr[800:1200, 800:1400]

In [ ]:
arr2[20,30]

In [ ]:
plt.imshow(arr2)
plt.show()

# environmental constraints (resistance)

In [ ]:
with rasterio.open(os.path.join(path_files, "corridor/COSTSURFACE.tif"), 'r') as ds:
    arr = ds.read()
print("read in cost array", arr.shape)
# cost_img = Image.fromarray(arr[0])
costs = normalize(arr[0])

In [ ]:
hard_cons = get_hard_constraints(os.path.join(path_files, "corridor"),os.path.join(path_files, "hard_constraints"))

### node constraints

In [ ]:
costs_rest = (costs*hard_cons.astype(int)) # [:, 500:2000]
costs_rest = reduce_instance(costs_rest, 16)
# todo: ensure sufficient zero padding

In [ ]:
plt.imshow(costs)
plt.show()

In [ ]:
plt.imshow(costs_shifted)
plt.show()

In [ ]:
shifts = get_half_donut(15,25)

In [ ]:
1621/60

## Edge constraints: get line and make convolutions

In [ ]:
shifts = get_half_donut(2.5,5)

In [ ]:
def bresenham_line(x0, y0, x1, y1):
    steep = abs(y1 - y0) > abs(x1 - x0)
    if steep:
        x0, y0 = y0, x0
        x1, y1 = y1, x1

    switched = False
    if x0 > x1:
        switched = True
        x0, x1 = x1, x0
        y0, y1 = y1, y0

    if y0 < y1:
        ystep = 1
    else:
        ystep = -1

    deltax = x1 - x0
    deltay = abs(y1 - y0)
    error = -deltax / 2
    y = y0

    line = []
    for x in range(x0, x1 + 1):
        if steep:
            line.append([y, x])
        else:
            line.append([x, y])

        error = error + deltay
        if error > 0:
            y = y + ystep
            error = error - deltax
    if switched:
        line.reverse()
    return line


In [ ]:
# lines = []
# for shift in shifts:
#     lines.extend(bresenham_line(0,0, shift[0],shift[1]))
# lines = np.array(lines)
lines = np.asarray(bresenham_line(0,0, 15,-6))
arr = np.zeros((16,16))
for l in lines:
    arr[l[0], l[1]+6] = 1
# plt.scatter(lines[:,0], lines[:,1])
plt.imshow(arr)
plt.show()

In [ ]:
def get_kernel(lower, upper):
    """
    Get all kernels describing the path of the edges in a discrete raster
    :param lower: minimum distance of towers
    :param upper: maximum distance of towers
    
    :returns kernel: all possible kernels (number of circle points x upper x upper)
    :returns posneg: a list indicating whether it is a path to the left (=1) or to the right(=0)
    """
    shifts = get_half_donut(lower, upper)
    posneg = []
    kernel = np.zeros((len(shifts), upper, upper))
    
    for i, shift in enumerate(shifts):
        if shift[1]<0:
            posneg.append(1)
            line = bresenham_line(0, upper-1, shift[0],upper-1+shift[1])
        else:
            posneg.append(0)
            line = bresenham_line(0, 0, shift[0],shift[1])
        # add points of line to the kernel
        for (j,k) in line:
            kernel[i,j,k] += 1
    return kernel, posneg

In [ ]:
kernel, posneg = get_kernel(2.5,5)

In [ ]:
def convolve(img, kernel, neg=0):
    k_size = len(kernel)
    if neg:
        padded = np.pad(img, ((0, k_size-1),(k_size-1,0)))
    else:
        padded = np.pad(img, ((0,k_size),(0,k_size)))
    # print(padded.shape)
    convolved = np.zeros(img.shape)
    w,h = img.shape
    for i in range(0, w):
        for j in range(0, h):
            patch = padded[i:i+k_size, j:j+k_size]
            convolved[i,j] = np.sum(patch*kernel)
    return convolved

# def convolve_negative(img, kernel):
#     k_size = len(kernel)
#     padded = np.pad(img, ((0, k_size-1),(k_size-1,0)))
#     # print(padded.shape)
#     convolved = np.zeros(img.shape)
#     w,h = img.shape
#     for i in range(0, w):
#         for j in range(0, h):
#             patch = padded[i:i+k_size, j:j+k_size]
#             convolved[i,j] = np.sum(patch*kernel)
#     return convolved

In [ ]:
arr = np.arange(0,81,1).reshape((9,9))
arr

In [ ]:
# example for negative kernel
kernel[14]

In [ ]:
from graph_tool.all import *

In [ ]:
arr2 = convolve(arr, kernel[14], actual[14])
print(arr2)

In [ ]:
# example for positive kernel
kernel[18]

In [ ]:
arr2 = convolve(arr, kernel[18])
print(arr2)

### Attempts to use scipy --> idea: use tensorflow?

In [ ]:
from scipy.signal import convolve2d

In [ ]:
arr = np.arange(0,20,1).reshape((4,5))
kernel = np.zeros((4,4))
kernel[[0,1,2],[0,1,2]] = 1

In [ ]:
arr2 = convolve2d(arr, kernel, mode ="same")

In [ ]:
arr

In [ ]:
arr2

### Test edge constraint optimization

In [ ]:
costs_small = reduce_instance(costs,16)

In [ ]:
dists = (3,6)
kernels, posneg = get_kernel(dists[0], dists[1])

In [ ]:
plt.imshow(costs_small)
plt.show()

In [ ]:
a = np.random.randint(len(kernels))
print(a)
print(kernels[a])
out = convolve(costs_small, kernels[a], posneg[a])

In [ ]:
plt.imshow(out)
plt.show()

In [ ]:
test_arr = np.zeros((20,20))
test_arr[10, 10] = 1
test_arr[11,10] = 1
out = convolve(test_arr, kernels[a], posneg[a])
plt.imshow(out)
plt.show()

# Optimize the computation of node constraints

In [ ]:
def get_shift_transformed(shifts):
    
    shift_tuples = []
    for shift in shifts:
        if shift[0]<0:
            tup1 = (0,-shift[0])
        else:
            tup1 = (shift[0],0)
        if shift[1]<0:
            tup2 = (0,-shift[1])
        else:
            tup2 = (shift[1],0)
        shift_tuples.append((tup1,tup2))
    
    return shift_tuples

In [ ]:
x_len, y_len = costs_rest.shape
node_pos = [(i, j) for i in range(x_len) for j in range(y_len) if costs_rest[i, j]]

pos2node = np.ones(costs_rest.shape)
pos2node *= -1
for n, (i,j) in enumerate(node_pos):
    pos2node[i,j] = n

In [ ]:
shift_tuples = get_shift_transformed(shifts)

orig_greater_zero = costs_rest>0
inds_orig = pos2node[costs_rest>0]

for i in range(1):
    print(shifts[i], shift_tuples[i])
    a, b, c, d = tuple(slice_tuples[i])
    costs_shifted = np.pad(costs_rest, shift_tuples[i], mode='constant')
    shift = shifts[i]
    if shift[0]>0 and shift[1]>0:
        costs_shifted = costs_shifted[:-shift[0], :-shift[1]]
    elif shift[0]>0 and shift[1]<=0:
        costs_shifted = costs_shifted[:-shift[0], -shift[1]:]
    elif shift[0]<=0 and shift[1]>0:
        costs_shifted = costs_shifted[-shift[0]:, :-shift[1]]
    elif shift[0]<=0 and shift[1]<=0:
        costs_shifted = costs_shifted[-shift[0]:, -shift[1]:]
    
    both_greater_zero = np.all(np.asarray([orig_greater_zero, costs_shifted>0]), axis=0)
    weights = (costs_shifted + costs_rest)/2
    
    inds_shifted = pos2node[costs_shifted>0]
    # delete the ones where inds_shifted is zero
    assert len(inds_shifted)==len(inds_orig)
    weights_list = weights[costs_shifted>0]
    
    pos_inds = inds_shifted>=0
    out = np.swapaxes(np.asarray([inds_orig, inds_shifted, weights_list]), 1,0)[pos_inds]
    print(out.shape)
    print(out[:100])
    # todo: problem with one is zero, other one not
    
    # call add_edge_list 600 times?
    
    plt.figure(figsize=(20,10))
    # plt.imshow((both_greater_zero.astype(int)+orig_greater_zero.astype(int)))
    plt.show()
    print()

In [ ]:
(i,j) = node_pos[208]
(k,l) = node_pos[224]
print(i,j,k,l)
print(costs_rest[i,j], costs_rest[k,l])

# old stuff

In [ ]:
cuts = []

    for shift in shifts:
        slice_tuple = []
        if shift[0]<0:
            slice_tuple.append(-shift[0])
            slice_tuple.append(None)
        else:
            slice_tuple.append(0)
            slice_tuple.append(-shift[0])
        if shift[1]<0:
            slice_tuple.append(-shift[1])
            slice_tuple.append(None)
        else: 
            slice_tuple.append(0)
            slice_tuple.append(-shift[1])
        cuts.append(slice_tuple)
    

### Open questions:

* How to combine different costs? simply add up the absolute values? Or normalize to 0-1 and weight?

### Possible constraints:

* Height of tower --> make six nodes instead of one, connect for each possible combination of heights
* Height of cables above ground
* Angles
* cables passing over environmental


### Todo:

* write possibility to split one surface into several parts and merge the paths in the end

In [ ]:
arr = np.zeros((10,10))
arr[3:5,4:7]=1

In [ ]:
np.where(arr>0)

In [ ]:
new_arr = np.zeros((10,10))
new_arr[np.where(arr>0)]=1

In [ ]:
new_arr

# Random tests

In [ ]:
arr2 = np.zeros((5,5))
arr2[2:5, 1:3]=1

In [ ]:
counter=0
for i in range(5):
    for j in range(5):
        arr[i,j] = counter
        counter+=1

In [ ]:
arr

In [ ]:
arr[arr2.astype(bool)]

In [ ]:
arr2